In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
import requests
import pandas as pd
import datetime as dt

In [2]:
url = 'https://www.scandichotels.com/'

path = ('C:/Users/Marijana/Downloads/chromedriver.exe')
s = Service(path)

driver = webdriver.Chrome(service=s)

driver.get(url)

driver.maximize_window()

agree = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//a[@id="CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll"]')))
try:
    agree.click()
except:
    pass

destination = driver.find_element(By.ID, 'widget-search__text')
destination.click()
destination.send_keys('Helsinki')
time.sleep(1)

hotel = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//li[@class="widget-search__list-item"]//span[text()="scandic paasi"]')))
hotel.click()

check_rates = driver.find_element(By.XPATH, '//button[@type="submit"]//span[text()="See rooms and prices"]')
check_rates.click()
time.sleep(3)

current_url = driver.current_url  

In [3]:
current_url

'https://www.scandichotels.com/hotelreservation/select-rate?hotel=634&fromDate=2023-03-31&toDate=2023-04-1&room%5b0%5d.adults=1'

In [4]:
response = requests.get(current_url)

In [5]:
response.status_code

200

In [6]:
soup = BeautifulSoup(response.content, 'html.parser')

In [7]:
results = soup.find_all('div', {'data-ng-controller': 'BookingRoomsAndRatesLegController'})

In [8]:
#Room type
results[0].find('h2', {'class': 'room__heading-level1'}).get_text()

'Standard Single'

In [9]:
#Rate
results[0].find('span', {'class': 'price'}).get_text()

'124 EUR/night'

In [10]:
room_type = []
rates = []

for result in results:
    
    #Room type
    try:
        room_type.append(result.find('h2', {'class': 'room__heading-level1'}).get_text())
    except:
        room_type.append('')
        
    #Rates
    try:
        rates.append(result.find('span', {'class': 'price'}).get_text())
    except:
        rates.append('')
                

In [11]:
df = pd.DataFrame(data=rates, index=room_type).T

In [12]:
today  = dt.date.today().strftime("%d/%m/%Y")
df['Date'] = today

In [13]:
#df.to_excel(dt.datetime.now().strftime("%d-%m-%Y") + '.xlsx', index=False)